# Anthropic Claude API Tutorial

This comprehensive tutorial covers essential concepts for working with Anthropic's Claude API, including the latest Claude 4 models.

## What You'll Learn
- Setup and authentication
- Basic text generation with Claude 4 models
- Streaming responses for real-time interaction
- Vision capabilities for image analysis
- Tool use (function calling)
- Async operations for performance
- Error handling and best practices

## Claude 4 Models Overview

**Claude 4 Opus** (`claude-opus-4-20250514`): Most capable model for complex reasoning
- $15/$75 per million tokens (input/output)
- Best for: Complex coding, research, advanced analysis

**Claude 4 Sonnet** (`claude-sonnet-4-20250514`): Balanced performance and efficiency
- $3/$15 per million tokens (input/output) 
- Best for: Production applications, coding, general tasks

**Claude 3.5 Haiku** (`claude-3-5-haiku-20241022`): Fastest and most cost-effective
- $0.80/$4 per million tokens (input/output)
- Best for: Real-time chat, content moderation, simple tasks

## 1. Setup and Installation

In [ ]:
# Install required packages
!pip install anthropic python-dotenv pillow

## 2. Authentication and Basic Setup

In [ ]:
import os
import anthropic
import base64
import asyncio
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Initialize client with API key
client = anthropic.Anthropic(
    api_key=os.environ.get("ANTHROPIC_API_KEY")
    # If no env var, you can pass directly: api_key="your-key-here"
)

print("Claude API client initialized successfully!")

## 3. Basic Text Generation with Claude 4

In [ ]:
def basic_chat(prompt, model="claude-sonnet-4-20250514"):
    """Basic text generation with Claude"""
    try:
        message = client.messages.create(
            model=model,
            max_tokens=1000,
            temperature=0.7,
            system="You are a helpful AI assistant.",
            messages=[
                {"role": "user", "content": prompt}
            ]
        )
        return message.content[0].text
    except Exception as e:
        return f"Error: {str(e)}"

# Example usage
response = basic_chat("Explain machine learning in simple terms")
print(response)

## 4. Streaming Responses for Real-time Interaction

Streaming reduces perceived latency by showing responses as they're generated.

In [ ]:
def stream_response(prompt):
    """Stream Claude's response in real-time"""
    try:
        with client.messages.stream(
            model="claude-sonnet-4-20250514",
            max_tokens=1000,
            messages=[{"role": "user", "content": prompt}]
        ) as stream:
            for text in stream.text_stream:
                print(text, end="", flush=True)
    except Exception as e:
        print(f"Streaming error: {e}")

# Example usage
print("Streaming response:")
stream_response("Write a short poem about artificial intelligence")
print("\n\nStreaming complete!")

## 5. Vision Capabilities - Image Analysis

Claude can analyze images and extract information from visual content.

In [ ]:
def analyze_image(image_path, prompt="Describe this image in detail"):
    """Analyze an image using Claude's vision capabilities"""
    try:
        # Read and encode image
        with open(image_path, "rb") as image_file:
            image_data = base64.b64encode(image_file.read()).decode()
        
        message = client.messages.create(
            model="claude-sonnet-4-20250514",
            max_tokens=1000,
            messages=[
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "image",
                            "source": {
                                "type": "base64",
                                "media_type": "image/jpeg",
                                "data": image_data
                            }
                        },
                        {
                            "type": "text",
                            "text": prompt
                        }
                    ]
                }
            ]
        )
        return message.content[0].text
    except Exception as e:
        return f"Vision error: {str(e)}"

# Example usage (replace with actual image path)
# response = analyze_image("path/to/your/image.jpg", "What objects do you see?")
# print(response)
print("Vision analysis function ready (provide image path to use)")

## 6. Tool Use (Function Calling)

Enable Claude to call external functions and tools for enhanced capabilities.

In [ ]:
import json
from datetime import datetime

def get_weather(location):
    """Mock weather function"""
    return f"The weather in {location} is sunny, 22°C"

def calculate(expression):
    """Safe calculator function"""
    try:
        return str(eval(expression))
    except:
        return "Invalid expression"

def chat_with_tools(prompt):
    """Chat with Claude using tools"""
    tools = [
        {
            "name": "get_weather",
            "description": "Get current weather for a location",
            "input_schema": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "City name"
                    }
                },
                "required": ["location"]
            }
        },
        {
            "name": "calculate",
            "description": "Perform mathematical calculations",
            "input_schema": {
                "type": "object",
                "properties": {
                    "expression": {
                        "type": "string",
                        "description": "Mathematical expression"
                    }
                },
                "required": ["expression"]
            }
        }
    ]
    
    message = client.messages.create(
        model="claude-sonnet-4-20250514",
        max_tokens=1000,
        tools=tools,
        messages=[{"role": "user", "content": prompt}]
    )
    
    return message

# Example usage
response = chat_with_tools("What's the weather in Paris and calculate 15 * 7?")
print(f"Response type: {response.content[0].type}")
if hasattr(response.content[0], 'text'):
    print(f"Text: {response.content[0].text}")

## 7. Async Operations for Better Performance

Use async operations to handle multiple requests concurrently.

In [ ]:
from anthropic import AsyncAnthropic

# Initialize async client
async_client = AsyncAnthropic(
    api_key=os.environ.get("ANTHROPIC_API_KEY")
)

async def async_chat(prompt, model="claude-sonnet-4-20250514"):
    """Async chat function"""
    message = await async_client.messages.create(
        model=model,
        max_tokens=500,
        messages=[{"role": "user", "content": prompt}]
    )
    return message.content[0].text

async def multiple_requests():
    """Handle multiple requests concurrently"""
    prompts = [
        "Explain quantum computing",
        "What is blockchain?", 
        "Define machine learning"
    ]
    
    tasks = [async_chat(prompt) for prompt in prompts]
    responses = await asyncio.gather(*tasks)
    
    for i, response in enumerate(responses):
        print(f"Response {i+1}: {response[:100]}...\n")

# Run async example
print("Running concurrent requests...")
await multiple_requests()
print("All requests completed!")

## 8. Error Handling and Best Practices

In [ ]:
from anthropic import APIError, RateLimitError
import time

def robust_chat(prompt, max_retries=3):
    """Chat with comprehensive error handling"""
    for attempt in range(max_retries):
        try:
            message = client.messages.create(
                model="claude-sonnet-4-20250514",
                max_tokens=1000,
                messages=[{"role": "user", "content": prompt}]
            )
            return {
                "success": True,
                "response": message.content[0].text,
                "usage": message.usage.dict() if hasattr(message, 'usage') else None
            }
            
        except RateLimitError:
            wait_time = (2 ** attempt) + 1  # Exponential backoff
            print(f"Rate limit hit. Waiting {wait_time}s...")
            time.sleep(wait_time)
            
        except APIError as e:
            return {
                "success": False,
                "error": f"API Error: {str(e)}"
            }
            
        except Exception as e:
            return {
                "success": False,
                "error": f"Unexpected error: {str(e)}"
            }
    
    return {
        "success": False,
        "error": "Max retries exceeded"
    }

# Example usage
result = robust_chat("Explain the importance of error handling in APIs")
print(f"Success: {result['success']}")
if result['success']:
    print(f"Response: {result['response'][:200]}...")
    if result['usage']:
        print(f"Tokens used: {result['usage']}")
else:
    print(f"Error: {result['error']}")

## 9. Cost Optimization Tips

**Model Selection**: Choose the right model for your task
- Use Haiku for simple, fast responses
- Use Sonnet for balanced performance
- Use Opus only for complex reasoning tasks

**Prompt Caching**: Cache frequently used prompts (90% savings)
**Batch Processing**: Use batch API for 50% cost reduction on non-urgent tasks
**Token Management**: Monitor input/output token usage

In [ ]:
def estimate_cost(input_tokens, output_tokens, model="claude-sonnet-4-20250514"):
    """Estimate API costs"""
    pricing = {
        "claude-opus-4-20250514": {"input": 15, "output": 75},
        "claude-sonnet-4-20250514": {"input": 3, "output": 15},
        "claude-3-5-haiku-20241022": {"input": 0.8, "output": 4}
    }
    
    if model not in pricing:
        return "Unknown model"
    
    costs = pricing[model]
    input_cost = (input_tokens / 1_000_000) * costs["input"]
    output_cost = (output_tokens / 1_000_000) * costs["output"]
    total_cost = input_cost + output_cost
    
    return {
        "input_cost": f"${input_cost:.6f}",
        "output_cost": f"${output_cost:.6f}", 
        "total_cost": f"${total_cost:.6f}"
    }

# Example cost calculation
cost = estimate_cost(1000, 500, "claude-sonnet-4-20250514")
print(f"Cost estimate: {cost}")

print("\n=== Tutorial Complete! ===")
print("You now know the essentials of Claude API:")
print("✅ Authentication and setup")
print("✅ Text generation with Claude 4 models")
print("✅ Streaming for real-time responses")
print("✅ Vision capabilities for images")
print("✅ Tool use for enhanced functionality")
print("✅ Async operations for performance")
print("✅ Error handling and cost optimization")

## Summary and Next Steps

This tutorial covered the essential Claude API concepts. For advanced use cases, explore:

**Advanced Features**:
- Prompt caching for cost optimization
- Extended thinking mode with Claude 4
- Computer use capabilities
- Custom tool integrations

**Production Considerations**:
- Rate limiting and retry logic
- Monitoring and logging
- Security best practices
- Scalability patterns

**Resources**:
- [Official Documentation](https://docs.anthropic.com)
- [API Reference](https://docs.anthropic.com/en/api)
- [Anthropic Cookbook](https://github.com/anthropics/anthropic-cookbook)
- [Community Discord](https://discord.gg/anthropic)

Happy building with Claude! 🚀